# EMI Metal Detector Design

## Design constraints
Mostly dictated by motor board. 
The goal being to reuse the motor board with minimum modifications to speed up the development.

### Shunt resistor
- R: 10m$\Omega$

### Op amp (INA282) 
- Bandwidth: 10kHZ
- Gain: 50
- Gain error: +/- 1.4%

### ADC low pass
- Cut-off: 4.7kHz

## Coil design

### Coil time constant
A coil time constant is given by
$$\tau = \frac{L}{R}$$

Given limiting sampling frequency: 4.7kHz, by Shannon we can measure well up to 500Hz signals.

Optimal coil time constant is 
$$\tau_{optimal} \approx \frac{1}{500} \approx 2ms$$

### Cylindrical coil inductance
A cylindrical coil inductance is given by
$$L = \frac{r^2 N^2}{20r + 28d}$$

where 
- L is the inductance in $\mu$H, 
- r is the mean radius in cm, 
- N is the number of turns, 
- d is depth of the coil (outer radius minus inner radius) in cm.

Source: Spiral air coil formula https://en.wikipedia.org/wiki/Inductor#Inductance_formulas

Converted to SI units ($H$ and $m$):
$$L = 10^{-4} \frac{r^2 N^2}{20r + 28d}$$

### Cylidrical coil resistance
Resistance is defined as
$$R = \frac{\rho l}{A} = \alpha l$$
with $\alpha = \frac{\rho}{A}$ the wire linear resistivity in $\Omega$/m, where
- $\rho$ is the resistivity in $\Omega$m
- l the length of the wire in m
- A the cross section of the wire in m$^2$

For a cylindrical coil, the wire length is given by the product of the cylinder perimeter and the number of turns:
$$l = 2 \pi r N$$

Such that the resistance of a cylindrical coil is
$$R = 2 \pi \alpha r N $$

### Coil time constant simplification
Plugging the formulas from above in the RL circuit time constant we get
$$\tau = \frac{L}{R} = \frac{10^{-4}}{2\pi} \frac{r N}{\alpha (20r + 28d)}$$

For a big enough coil (r > 5cm), the depth of the coil $d$ is smaller than the coil radius $d << r$ so we can neglect in the formula, which yields the simplified version:
$$\tau \approx \frac{10^{-4}}{2\pi} \frac{r N}{20\alpha r} = \frac{10^{-5}}{4\pi} \frac{N}{\alpha}$$

The time constant now only depends on the number of turns $N$ and the linear resistivity of the wire $\alpha$.
These are our two design parameters to match the desired $\tau_{optimal}$.

### Fixing other parameters
The depth of the coil should be kept minimal (<1cm) given the simplification done above.

The radius of the coil determines what size of objects our detector will sense best.
We want to detect Anti-Personal landmine size objects which range from 6cm to 14cm. So our coil should be in 14-30cm diameter range, so $r \in [0.07, 0.15]$.

In [21]:
%matplotlib widget
from math import *
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

AWG_LINEAR_RESISTIVITY = {'0': 0.00032, '1': 0.0004, '2': 0.0005, '3': 0.00064, '4': 0.0008, '5': 0.001, '6': 0.00125, '7': 0.0016, '8': 0.002, '9': 0.0025, '10': 0.0032, '11': 0.004, '12': 0.005, '13': 0.0064, '14': 0.008, '15': 0.01, '16': 0.0125, '17': 0.016, '18': 0.02, '19': 0.025, '20': 0.032, '21': 0.04, '22': 0.05, '23': 0.064, '24': 0.08, '25': 0.1, '26': 0.125, '27': 0.16, '28': 0.2, '29': 0.25, '30': 0.32, '31': 0.4, '32': 0.5, '33': 0.64, '34': 0.8, '35': 1, '36': 1.25, '37': 1.6, '38': 2, '39': 2.5}

In [22]:
def coil_inductance(r, N, d):
    return 1e-4 * ((r**2) * (N**2)) / (20 * r + 28 * d)

def coil_length(r, N):
    return 2 * pi * r * N

def coil_resistance(r, N, alpha):
    return coil_length(r, N) * alpha

def coil_turn_count(tau, alpha): 
    return (4 * pi) * (1e5) * tau * alpha

def design_coil(tau, awg, r = 0.07, d = 0.01):
    alpha = AWG_LINEAR_RESISTIVITY[str(awg)]
    N = round(coil_turn_count(tau, alpha))
    L = coil_inductance(r, N, d)
    R = coil_resistance(r, N, alpha)
    return N, L, R, L/R, r

In [54]:
def show_coil(tau_ms, awg, r_cm = 7, d_cm = 1):
    N, L, R, tau, r = design_coil(tau_ms/1000, awg, r_cm/100, d_cm/100)
    print('Coil with     {} turns'.format(N))
    print('Inductance    {:.7f} H'.format(L))
    print('Resistance    {:.7f} Ohm'.format(R))
    print('Tau           {:.7f} s'.format(tau))
    print('You will need {:.7f} m of wire'.format(coil_length(r, N)))

interact(show_coil, 
         tau_ms=fixed(2), 
         awg=widgets.Dropdown(options=AWG_LINEAR_RESISTIVITY.keys(), value='24', description='AWG'),
         r_cm=widgets.BoundedFloatText(value=7.0, min=1.0, max=30.0, description='Radius [cm]'),
         d_cm=widgets.BoundedFloatText(value=0.5, min=0.1, max=2.0, step=0.1, description='Depth [cm]'));

interactive(children=(Dropdown(description='AWG', index=24, options=('0', '1', '2', '3', '4', '5', '6', '7', '…